In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.applications import ResNet50

In [ ]:
path = '/kaggle/input/happy-whale-and-dolphin/'
os.listdir(path)

In [ ]:
train_data = pd.read_csv(path+'train.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

In [ ]:
train_data.individual_id.unique

In [ ]:
train_data

In [ ]:
samp_subm.loc[0, 'predictions']

In [ ]:
print('Number train samples:', len(train_data))
print('Number train images:', len(os.listdir(path+'train_images/')))
print('Number test images:', len(os.listdir(path+'test_images/')))

In [ ]:
train_data.head()

In [ ]:
train_data['species'].value_counts()


In [ ]:
train_data['individual_id'].value_counts()


In [ ]:
row = 0
file = train_data.loc[row, 'image']
species = train_data.loc[row, 'species']

img = cv2.imread(path+'train_images/'+file)
print('Shape:', img.shape)

In [ ]:
img[1]

In [ ]:
def image_preprocessing(image, image_size):
    """ Image Preprocessing """
    
    # Crop Image
    mid_row = int(image.shape[0]/2)
    mid_col = int(image.shape[1]/2)
    if image.shape[0]>image.shape[1]:
        image_cropped = image[mid_row-mid_col:mid_row+mid_col,
                                   0:image.shape[1]]
    else:
        image_cropped = image[0:image.shape[0],
                                   mid_col-mid_row:mid_col+mid_row]
    
    # Rescale Image
    image_rescale = cv2.resize(image_cropped,
                               dsize=(image_size, image_size))
    return image_rescale


def plot_befor_after(image):
    """ Compare original and prepared image """
    
    fig, axs = plt.subplots(1, 2, figsize=(15, 10))
    fig.subplots_adjust(hspace = .1, wspace=.1)
    axs = axs.ravel()
    # Plot Original Image
    axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[0].set_title('original shape: '+str(image.shape))
    # Image Preprocessing
    image_rescale = image_preprocessing(image, image_size)
    # Plot Prepared Image
    axs[1].imshow(cv2.cvtColor(image_rescale, cv2.COLOR_BGR2RGB))
    axs[1].set_title('rescaled shape: '+str(image_rescale.shape))
    for i in range(2):
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
    plt.show()
image_size = 128
row = 2022
file = train_data.loc[row, 'image']
species = train_data.loc[row, 'species']
image = cv2.imread(path+'train_images/'+file)
print('Shape:', image.shape)

In [ ]:
train_data.individual_id.unique()

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input

from tqdm.autonotebook import tqdm
def Loading_Images(data, m, dataset):
    print("Loading images")
    X_train = np.zeros((m, 32, 32, 3))
    count = 0
    for fig in tqdm(data['image']):
        if count>5:
            img = image.load_img("../input/happy-whale-and-dolphin/"+dataset+"/"+fig, target_size=(32, 32, 3))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            X_train[count] = x
            count += 1

    return X_train

In [ ]:
loaded = Loading_Images(train_data, train_data.shape[0], "train_images")
#X /= 255

In [ ]:
loaded.shape

In [ ]:
train_data.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    y = onehot_encoded
    return y, label_encoder

In [ ]:
y, label_encoder = prepare_labels(train_data['individual_id'])
y.shape

In [ ]:
plot_befor_after(img[1])

In [ ]:

X =loaded/ 255

In [ ]:
np.save("X.npy", X)

np.save("y.npy", y)


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.5,random_state = 1)
# Normalize pixel values to be between 0 and 1


In [ ]:
del X
del train_data
#del yo

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
"""model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))"""

In [ ]:
"""model.summary()"""

In [ ]:
"""model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))"""

In [ ]:
"""model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))"""

In [ ]:
"""model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_test, y_test))"""

In [ ]:
import keras.backend as K
from keras.models import Sequential
from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model

model = Sequential()

model.add(Conv2D(32, (6, 6), strides = (1, 1), input_shape = (32, 32, 3)))
model.add(BatchNormalization(axis = 3))
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), strides = (1,1)))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3)))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.85))
model.add(Dense(712, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=1, batch_size=128, verbose=1)
